In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pypsa
import seaborn as sns

In [2]:
EXPORT = 1
CO2L = 2.0
CLUSTERS = 4
H = 3
LL = "3.0"
DISCOUNT_RATE = 0.13

RUN = "decr_14_3h_ws"
CATEGORY = "postnetworks"
SCENARIO = f"elec_s_{CLUSTERS}_ec_lc{LL}_Co2L{CO2L}_{H}H_2030_{DISCOUNT_RATE}_DF_{EXPORT}export"

#OVERRIDES = PATH + "pypsa-eur-sec/data/override_component_attrs"

PREFIX = "/mnt/c/Users/scl38887/Documents/git/aldehyde/"
OUTPUT = PREFIX + "results"
INPUT = PREFIX + f"workflow/subworkflows/pypsa-earth-sec/results/{RUN}/{CATEGORY}/{SCENARIO}.nc"
OUTPUT_SCENARIO = f"{OUTPUT}/{RUN}/graphics/demand/{SCENARIO}"
OUTPUT_RAW = f"{OUTPUT}/{RUN}/graphics/demand/"

In [3]:
n = pypsa.Network(INPUT)

INFO:pypsa.io:Imported network elec_s_4_ec_lc3.0_Co2L2.0_3H_2030_0.13_DF_1export.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


### Marginal prices of all carriers

In [26]:
def get_bus_demand(n, busname, carrier_limit=False, carrier_limit_integration=False):
    """Get the demand at a certain bus (includes stores/StorageUnits) based on the energy_balance() function

    Parameters
    ----------
    busname : _type_
        _description_
    n : _type_
        _description_
    """

    # Get the energy balance of the bus specified in busname
    if carrier_limit == False:
        energy_balance_bus = energy_balance.loc[:, :, :, busname]
    else:
        if carrier_limit_integration == "inclusive":
            energy_balance_bus = energy_balance.loc[:,carrier_limit, :, busname]

        elif carrier_limit_integration == "exclusive":  # exclude carrier_limit
            energy_balance_bus = energy_balance.loc[:, :, :, busname]
            # Create a boolean mask to exclude rows with 'carrier_limit' in the third level
            condition = energy_balance_bus.index.get_level_values(1) != carrier_limit
            energy_balance_bus = energy_balance_bus[condition]  
        else:
            logging.error("carrier_limit_integration must be 'inclusive' or 'exclusive'")

    # Filter for negative values, filter out Stores/StorageUnits, filter out export links "H2" which are not in "H2 export buses", and sum them up. Inclues H2 pipeline
    demand = energy_balance_bus[energy_balance_bus < 0]
    demand = demand[(demand.index.get_level_values(0) != "Store") & (demand.index.get_level_values(0) != "StorageUnit")]
    demand = demand[(demand.index.get_level_values(1) != "H2") | (demand.index.get_level_values(3) == "H2 export bus")]
    demand = demand.groupby("bus").sum()

    #only when demand.index.get_level_values(1) == "H2 export bus" dann wird gekickt

    return demand.transpose()

In [27]:
energy_balance = n.statistics.energy_balance(aggregate_bus=False, aggregate_time=False)
carrier = "AC"
buses_sel = n.buses[n.buses.carrier == carrier].index
demand = get_bus_demand(n, buses_sel, carrier_limit="H2 Electrolysis", carrier_limit_integration="exclusive")

In [28]:
demand.sum().sum() * n.snapshot_weightings.generators[0] / 1e6

-90.27539911876887

In [22]:
loads = n.loads_t.p.groupby(n.loads.carrier, axis=1).sum()
ac_buses = n.buses[n.buses.carrier == "AC"]
ac_loads = n.loads.loc[n.loads.bus.isin(ac_buses.index)].carrier.unique()
# Sum of demand through loads on AC buses
el_base_demand = (loads.loc[:, ac_loads].sum() * n.snapshot_weightings.generators[0] / 1e6).sum().round(2) # TWh

In [23]:
el_base_demand

55.37

### Test energy balance stats

In [11]:
n.statistics.dispatch(bus_carrier="AC")

component    carrier                                     
StorageUnit  Pumped Hydro Storage                           -6.789124e+01
             Reservoir & Dam                                 1.021541e+07
Link         DAC                                            -6.204297e+01
             H2 Electrolysis                                -2.160430e+06
             H2 Fuel Cell                                    7.923148e+01
             Open-Cycle Gas                                  2.907694e+02
             battery charger                                -2.692366e+03
             battery discharger                              2.584672e+03
             biomass EOP                                     1.466495e+01
             helmeth                                        -1.111325e+02
             residential rural ground heat pump             -1.098944e+02
             residential rural resistive heater             -3.022088e+02
             residential urban decentral air heat pump

### Average nodal prices of all carriers

In [10]:
def get_nodal_prices(n):
    return (
        n.buses_t.marginal_price.mean()
        .groupby([n.buses.location, n.buses.carrier])
        .first()
        .unstack()
    )

In [13]:
def get_mean_prices(n):
    return (
        n.buses_t.marginal_price.mean()
        .groupby([n.buses.carrier])
        .first()
    )

In [12]:
df = get_nodal_prices(n)
df

carrier
AC                                          54.764191
H2                                          90.253097
Li ion                                      52.280194
battery                                     54.771567
biogas                                      72.662175
co2                                       -457.453221
co2 stored                                -252.124248
gas                                        -15.637795
gas for industry                            54.202242
oil                                         50.010045
process emissions                         -322.062878
residential rural heat                       8.600139
residential rural water tanks               10.250459
residential urban decentral heat            10.078868
residential urban decentral water tanks     12.009825
services rural heat                          8.601819
services rural water tanks                  10.252680
services urban decentral heat               32.136573
services urban decen